In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime,timedelta
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import scipy.optimize
import mysql.connector
import pytz

c:\Users\igorb\anaconda3\lib\site-packages\mpl_finance.py:16: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  __warnings.warn('\n\n  ================================================================='+


In [3]:
conn = mysql.connector.connect(
  host="localhost",
  user="root",
  password="1234",
  database="bolsa_valores"
)
cursor = conn.cursor()

x = pd.read_sql("SELECT * FROM cotacao", conn)
x

C:\Users\igorb\AppData\Local\Temp\ipykernel_11732\2082163194.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  x = pd.read_sql("SELECT * FROM cotacao", conn)


,DATE,SYMBOL,OPEN,HIGH,LOW,CLOSE,VOLUME,DIVIDENDS
0,2020-03-23,ABEV3,10.886105,10.886105,9.910966,10.203507,50158000,0.0
1,2020-03-23,ALPA4,21.352230,23.023619,20.274234,23.023619,3632400,0.0
2,2020-03-23,ALSO3,24.285420,25.054070,20.364338,21.327581,2269200,0.0
3,2020-03-23,ARZZ3,33.663654,34.227903,31.368404,32.076107,798500,0.0
4,2020-03-23,AZUL4,14.220000,14.340000,12.530000,14.280000,13001300,0.0
...,...,...,...,...,...,...,...,...
70571,2023-03-23,VBBR3,13.520000,13.600000,12.770000,12.930000,11172700,0.0
70572,2023-03-23,VIIA3,2.000000,2.010000,1.760000,1.850000,109159900,0.0
70573,2023-03-23,VIVT3,38.459999,39.410000,38.290001,38.529999,2380300,0.0
70574,2023-03-23,WEGE3,39.599998,40.360001,39.529999,39.889999,6448400,0.0


In [4]:
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
df_final = pd.DataFrame()
for ativo in symbol_list_ibrx:
    chamada_api = yf.Ticker(ativo + ".SA").history(period='36mo')
    df_hist = pd.DataFrame(chamada_api).reset_index()
    df_hist["Symbol"] = ativo
    df_hist = df_hist[["Date","Symbol","Open","High","Low","Close","Volume","Dividends"]]
    df_hist["Date"] = df_hist["Date"].dt.tz_convert(pytz.utc).dt.date
    df_hist["Date"] = df_hist["Date"].astype(str)
    df_final = pd.concat([df_hist, df_final],axis=0)

tuplas = tuple(df_final.itertuples(index=False, name=None))
query = "INSERT IGNORE INTO cotacao  VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(query, tuplas)
conn.commit()
conn.close()